<h1>Inference Using ollama-complex Model Interface</h1>
This notebook probes how "ollama-complex" in model_interfaces.py works on the Watchman dataset images.</br>
It requires "dataset" service to collect a number of images triggering true and false positiveves.</br>
See fine-tuning section in the readme.txt in this folder for the details about the "dataset" service.</br>


In [ ]:
import os
import sys
import base64
from pathlib import Path
import pandas as pd

In [ ]:
# set up vars for finding train data
dataset = "../.data/dataset" # dataset folder location
chans = ["porch"] # list of channels to load the data for
objs = ["person"] # list of objects to load the data for
model_name = "ollama-complex" # model interface name to use for inference experimentation
c_desc = {
    "porch": "Porch",
}
o_desc = {
    "person": "a person",
}

In [ ]:
# load model interfaces we can use for testing
print(f"Working dir: {os.getcwd()}")
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../orchestrator"))
from shared_settings import *
from model_interfaces import *
print(MODELS)

In [ ]:
# create an instance of the model_name interface
MODEL_INTERFACE = MODELS[model_name]()

In [ ]:
# run test inference
from PIL import Image
from IPython.display import display

def test_inf(s, c, o, res):
    print(f"Subdir: {s} Expecting: {res}")
    image_pname = f"{s}/image.jpg"
    img_data = base64.b64encode(Path(image_pname).read_bytes()).decode()
    res, msg = MODEL_INTERFACE.locate(img_data, o_desc[o], c_desc[c])
    img = Image.open(image_pname)
    w, h = img.size
    display(img.resize((int(w / 4), int(h / 4))))
    print("Inference result: ", "yes" if res else "no")
    print("Location: ", msg if msg is not None else "N/A")
    print("---------------------------")

for c in chans:
    for o in objs:
        dir = f"{dataset}/{c}/{o}"
        subdirs = [f.path for f in os.scandir(dir) if f.is_dir()]
        true_pos = False
        false_pos = False
        for s in subdirs:
            if true_pos and false_pos:
                break
            if os.path.exists(f"{s}/skip"):
                continue
            if not false_pos and os.path.exists(f"{s}/no"):
                test_inf(s, c, o, "no")
                false_pos = True
            if not true_pos and not os.path.exists(f"{s}/no"):
                test_inf(s, c, o, "yes")
                true_pos = True
            